In [1]:
import math
import copy
import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dim=512, num_heads=8):
        super().__init__()
        assert model_dim % num_heads == 0, "model_dim은 num_heads로 나누어떨어져야 한다."

        # 입력받은 값을 인스턴스 변수로 저장
        self.model_dim = model_dim
        self.num_heads = num_heads
        self.key_dim = model_dim // num_heads ## d_k = d_model // num_heads

        ## num_heads개의 Q, K, V
        ## 연산의 효율성을 높히기 위해 한 번에 num_heads개의 Q, K, V를 만들어 낸다.
        self.query_linear = nn.Linear(model_dim, model_dim)
        self.key_linear = nn.Linear(model_dim, model_dim)
        self.value_linear = nn.Linear(model_dim, model_dim)

        ## 출력층
        self.output_linear = nn.Linear(model_dim, model_dim)

    # 입력 Tensor를 num_heads 만큼 분할.
    def split_heads(self, tensor):
        ## tensor : [batch_size, seq_len, d_model]
        batch_size, seq_length, model_dim = tensor.size()

        ## [batch_size, num_heads, seq_len, d_k]
        return tensor.view(batch_size, seq_length, self.num_heads, self.key_dim).transpose(1, 2)
    
    # 분할된 헤드를 다시 합침
    def combine_heads(self, tensor):
        batch_size, _, seq_length, key_dim = tensor.size()
        return tensor.transpose(1, 2).contiguous().view(batch_size, seq_length, self.model_dim)

    def scaled_dot_product_attention(self, query, key, value, mask=None):
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) ## Q \cdot K^T
        attention_scores = attention_scores / math.sqrt(self.key_dim) ## \sqrt d_k로 scaling

        ## Decoder의 self-attentention에서 현재시점 t부터 미래 값들을 마스크로 가리기 위함.
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)

        attention_dist = torch.softmax(attention_scores, dim=-1)
        output = torch.matmul(attention_dist, value)

        return output
    
    def forward(self, query, key, value, mask=None):
        ## q, k, v : [batch_size, seq_len, d_model]

        ## Query, Key, Value에 각각 선형 변환을 적용하고 헤드를 분할
        ## [batch_size, num_heads, seq_len, d_k]
        query = self.split_heads(self.query_linear(query))
        key = self.split_heads(self.key_linear(key))
        value = self.split_heads(self.value_linear(value))

        ## Scaled Dot-Product Attention을 계산
        ## (batch_size, num_heads, seq_length, seq_length)
        attention_output = self.scaled_dot_product_attention(query, key, value, mask)

        ## 분할된 헤드를 다시 합치고, 선형 변환을 적용
        ## (batch_size, seq_length, model_dim)
        output = self.output_linear(self.combine_heads(attention_output))
        
        return output    

In [4]:
class FeedForwardNN(nn.Module):
    def __init__(self, model_dim, feedforward_dim):
        super().__init__()
        self.fc1 = nn.Linear(model_dim, feedforward_dim)
        self.fc2 = nn.Linear(feedforward_dim, model_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

##### Positional Encoding 구현

Positional Encoding은 Transformer 모델에서 입력 시퀀스의 위치 정보를 제공하는 메커니즘입니다. 이를 통해 모델이 각 입력 단어의 위치에 대한 정보를 인식하고 처리할 수 있습니다

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/4cea42db-d3dc-4214-a9c3-c4580c35dfa1" height="550">

`PositionalEncoding` 클래스는 다음과 같이 구현됩니다:

- `__init__` 함수에서는 `model_dim`과 `max_seq_length`를 인자로 받습니다. `model_dim`은 모델의 차원을, `max_seq_length`는 최대 시퀀스 길이를 나타냅니다. 이 함수에서는 `positional_encoding` 텐서를 초기화하고, sin과 cos 함수를 사용하여 위치 정보를 인코딩합니다

- `forward` 함수에서는 입력 텐서 `x`와 `positional_encoding`을 더하여 위치 정보가 포함된 텐서를 반환합니다. 이렇게 함으로써, 모델은 입력 시퀀스의 각 단어에 대한 위치 정보를 인식할 수 있습니다

- 짝수 인덱스에 대한 수식:
$PE_{(pos, 2i)} = sin(pos / 10000^{2i/d_{model}})$

- 홀수 인덱스에 대한 수식:
$PE_{(pos, 2i+1)} = cos(pos / 10000^{2i/d_{model}})$

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/5a266edb-5006-407c-8ce2-ea6d34dc51ae" height="400">


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_seq_length):
        super().__init__()

        ## Positional Encoding을 계산하기 위한 빈 텐서 생성. (max_seq_length, model_dim)
        positional_encoding = torch.zeros(max_seq_length, model_dim)

        ## 각 위치(position)에 대한 정보를 담은 텐서를 생성. (max_seq_length, 1)
        ## 0부터 max_seq_length - 1까지의 정수 시퀀스를 생성.
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        ## 분모 계산
        ## 0부터 model_dim까지 2씩 증가하는 시퀀스를 생성.
        ## e^(2i * -log(10000) / d_model)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * -(math.log(10000.0) / model_dim))

        ## 짝수 인덱스에는 sin 함수를 적용
        ## 짝수 인덱스에 대한 수식: PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)

        ## 홀수 인덱스에는 cos 함수를 적용
        ## 홀수 인덱스에 대한 수식: PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
        positional_encoding[:, 1::2] = torch.cos(position * div_term)

        ## 계산된 Positional Encoding을 모듈의 버퍼에 등록
        ## 이를 통해 모듈의 state_dict에 포함되어 저장 및 로딩이 가능해짐.(학습 X)
        self.register_buffer('positional_encoding', positional_encoding.unsqueeze(0))

    def forward(self, x):
        ## 입력 x(embedding)와 Positional Encoding을 더하여 반환
        ## 이 때, 입력 x의 시퀀스 길이에 맞추어 Positional Encoding을 슬라이싱함
        return x + self.positional_encoding[:, :x.size(1)]

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, feedforward_dim, dropout):
        super().__init__()
        self.self_attention = MultiHeadAttention(model_dim, num_heads)
        self.feed_forward = FeedForwardNN(model_dim, feedforward_dim)

        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        ## Self-Attention을 계산하고, 결과를 원래의 입력과 더한 후(skip connection) Layer Normalization 적용
        attention_output = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attention_output))

        ## Feedforward를 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization 적용
        feedforward_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(feedforward_output))

        return x

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, feedforward_dim, dropout):
        super().__init__()
        ## Decoder Self-Attention(Casual attention)
        self.self_attention = MultiHeadAttention(model_dim, num_heads)
        
        ## Encoder-Decoder Attention(Cross attention)
        self.cross_attention = MultiHeadAttention(model_dim, num_heads)

        self.feed_forward = FeedForwardNN(model_dim, feedforward_dim)

        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.norm3 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        ## Self-Attention을 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        ## tgt_mask는 decoder self-attention에서 현재~미래 정보를 가리기 위함.
        attention_output = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attention_output))

        ## Cross-Attention을 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        attention_output = self.cross_attention(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(attention_output))

        ## Feedforward를 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        feedforward_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(feedforward_output))

        return x


In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, model_dim, num_heads, num_layers, feedforward_dim, max_seq_length, dropout):
        super().__init__()
        ## Embedding과 Positional Encoding 정의
        self.encoder_embedding = nn.Embedding(src_vocab_size, model_dim)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim, max_seq_length)

        ## Encoder와 Decoder 레이어를 정의
        self.encoder_layers = nn.ModuleList([EncoderLayer(model_dim, num_heads, feedforward_dim, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(model_dim, num_heads, feedforward_dim, dropout) for _ in range(num_layers)])

        ## output layer
        self.fc = nn.Linear(model_dim, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    ## 마스크를 생성하는 함수 (Decoder의 self-attention = casual attention)
    def generate_mask(self, src, tgt):
        ## 입력된 소스와 타겟에서 각각 0이 아닌 위치를 찾아 마스크를 생성
        ## 즉, 인코더의 입력에서 패딩 토큰이 학습에 영향을 미치지 않도록 패딩된 위치를 마스킹.
        ## src != 0은 입력 시퀀스에서 패딩(0)이 아닌 값들을 찾아 True로, 패딩된 부분은 False로 변환.
        ## attention 스코어와 연산을 할 수 있게 하기 위해, unsqueeze를 사용하여 차원을 추가
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(device)
        ## 여기까지는 패딩 토큰은 False, 그 이외에는 True인 마스크.

        ## 타겟의 시퀀스 길이를 가져옴
        seq_length = tgt.size(1)

        ## nopeak_mask는 디코더가 자신보다 미래의 단어를 참조하지 못하게 하는 마스크
        ## triu로 상삼각 행렬을 만든 후, 1 - triu를 통해 미래 시점에 해당하는 값을 0으로 바꾸고, 나머지 값들을 1로 유지하여 하삼각 행렬을 만든다.
        ## 즉, 대각선 아래쪽은 0, 위쪽은 1인 상삼각행렬을 생성하고, 이를 불리언 타입으로 변환
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        ## 이는 현재시점 t부터 미래시점을 모두 False로 마스킹.

        ## 타겟 마스크와 nopeak_mask를 AND 연산하여 최종 타겟 마스크를 생성
        ## 이 마스크는 디코더가 패딩 위치 뿐 아니라 자신보다 미래의 단어를 참조하지 못하게 함
        tgt_mask = tgt_mask & nopeak_mask

        ## 소스 마스크와 타겟 마스크를 반환
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        ## 마스크 생성
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        ## 소스와 타겟에 각각 Embedding과 Positional Encoding을 적용
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        ## Encoder를 통과 (Encoder 순전파)
        encoder_output = src_embedded
        for encoder_layer in self.encoder_layers:
            encoder_output = encoder_layer(encoder_output, src_mask)

        ## Decoder를 통과 (Decoder 순전파)
        decoder_output = tgt_embedded
        for decoder_layer in self.decoder_layers:
            decoder_output = decoder_layer(decoder_output, encoder_output, src_mask, tgt_mask)

        ## 최종 출력 계산
        output = self.fc(decoder_output)
        return output

In [9]:
# 하이퍼파라미터 설정
src_vocab_size = 5000
tgt_vocab_size = 5000
model_dim = 512
num_heads = 8
num_layers = 6
feedforward_dim = 2048
max_seq_length = 100
dropout = 0.1
num_steps = 5000

# Transformer 모델을 생성
transformer = Transformer(src_vocab_size, tgt_vocab_size, model_dim, num_heads, num_layers, feedforward_dim, max_seq_length, dropout).to(device)

# 학습용 랜덤 데이터 생성
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (배치 크기, 시퀀스 길이)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (배치 크기, 시퀀스 길이)

# 손실 함수와 최적화 알고리즘을 정의
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# 모델을 학습 모드로 설정
transformer.train()

# 100회의 에폭 동안 학습.
for epoch in range(100):
    # 기울기 초기화
    optimizer.zero_grad()
    # 모델의 순전파 실행
    output = transformer(src_data, tgt_data[:, :-1])
    # 손실 계산
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    # 역전파 실행
    loss.backward()
    # 가중치 업데이트
    optimizer.step()
    # 현재 에폭 및 손실 출력
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.68000316619873
Epoch: 2, Loss: 8.553583145141602
Epoch: 3, Loss: 8.481226921081543
Epoch: 4, Loss: 8.425561904907227
Epoch: 5, Loss: 8.36943244934082
Epoch: 6, Loss: 8.298233032226562
Epoch: 7, Loss: 8.213747024536133
Epoch: 8, Loss: 8.141448020935059
Epoch: 9, Loss: 8.053771018981934
Epoch: 10, Loss: 7.977225303649902
Epoch: 11, Loss: 7.892059803009033
Epoch: 12, Loss: 7.812472343444824
Epoch: 13, Loss: 7.731770038604736
Epoch: 14, Loss: 7.645899772644043
Epoch: 15, Loss: 7.565388202667236
Epoch: 16, Loss: 7.477782726287842
Epoch: 17, Loss: 7.398458957672119
Epoch: 18, Loss: 7.315279483795166
Epoch: 19, Loss: 7.2346625328063965
Epoch: 20, Loss: 7.15229606628418
Epoch: 21, Loss: 7.076003074645996
Epoch: 22, Loss: 6.994831562042236
Epoch: 23, Loss: 6.915043354034424
Epoch: 24, Loss: 6.839944839477539
Epoch: 25, Loss: 6.766018390655518
Epoch: 26, Loss: 6.702005386352539
Epoch: 27, Loss: 6.6314473152160645
Epoch: 28, Loss: 6.542862892150879
Epoch: 29, Loss: 6.47590017318